In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-27 06:22:05--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-27 06:22:05 (55.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-27 06:22:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-06-27 06:22:16 (97.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing dataset

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(f"Total training samples: {len(train_df)}")
print(f"Total testing samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total testing samples: 3263
Total samples: 10876


In [11]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
import random

In [13]:
random_index = random.randint(0, len(train_df)-5)

for index in range(random_index, random_index+5):
    print(f'Target: {train_df["target"][index]}')
    print(f'Text:\n{train_df["text"][index]}')
    print("\n")

Target: 1
Text:
lizzie363 @CstSmith I drove thru deep water in a flash flood--rainstorm repair cost $14000. 25 yrs later people still thnk cars float!!!


Target: 0
Text:
Best believe all the wrong decisions are being made out here in these Memphis streets during this here rainstorm lol my folk doe


Target: 0
Text:
I want it to rainstorm PLEASE


Target: 0
Text:
Waking up sick with a rainstorm outside would usually make me sad. Not today though. Put some The Kooks on the stereo and let's do this.


Target: 1
Text:
Yay I can feel the wind gearing up for a rainstorm in #Vancouver!! Bring it on #drought #deadgrassandflowers #wildfires




### Spliting Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
temp = pd.DataFrame(data=train_sentences[:10], columns=["text"])
temp["target"] = train_labels[:10]
temp

,text,target
0,@mogacola @zamtriossu i screamed after hitting...,0
1,Imagine getting flattened by Kurt Zouma,0
2,@Gurmeetramrahim #MSGDoing111WelfareWorks Gree...,1
3,@shakjn @C7 @Magnums im shaking in fear he's g...,0
4,Somehow find you and I collide http://t.co/Ee8...,0
5,@EvaHanderek @MarleyKnysh great times until th...,1
6,destroy the free fandom honestly,1
7,Weapons stolen from National Guard Armory in N...,0
8,@wfaaweather Pete when will the heat wave pass...,1
9,Patient-reported outcomes in long-term survivo...,1


### Text vectorization (tokenization)

In [18]:
from tensorflow.keras.layers import TextVectorization

In [19]:
avg_sentence_length = round(sum([len(sentence.split()) for sentence in train_sentences]) / len(train_sentences))
avg_sentence_length

15

In [20]:
# this method's every parameter is set by default, what we need. Just need to change below two params.
text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_sequence_length=avg_sentence_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
import random

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Before tokenization: {random_sentence}")
print(f"After tokenization: {text_vectorizer([random_sentence])}")

Before tokenization: Businesses are deluged with invoices. Make yours stand oup with colour or shame and it's likely to rise to the top of the pay' pile.
After tokenization: [[1037   22  655   14 1832  144  829  807    1   14 1423   53 1595    7
    37]]


In [24]:
print(f"5 most common words: {text_vectorizer.get_vocabulary()[:5]}")
print(f"5 least common words: {text_vectorizer.get_vocabulary()[-5:]}")

5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Embedding

In [25]:
from tensorflow.keras.layers import Embedding

In [26]:
embedding = Embedding(input_dim=len(text_vectorizer.get_vocabulary()),
                      output_dim=128,
                      input_length=avg_sentence_length)

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Sentence before embedding: {random_sentence}")
embedding(text_vectorizer([random_sentence]))

Sentence before embedding: I'm crazy enough to run in 95 degree mid-day heat under the blazing sun in a place where I'm notÛ_ https://t.co/OSUoIVNiGO


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02306705,  0.01659173,  0.02345039, ...,  0.02047563,
          0.02480134,  0.01821145],
        [ 0.01110127, -0.03468496,  0.03998586, ...,  0.03462159,
         -0.0486747 ,  0.04899177],
        [-0.00396104,  0.00055335,  0.00046055, ..., -0.03671402,
          0.00461901,  0.00175744],
        ...,
        [-0.00364759,  0.04568047,  0.02025065, ...,  0.04367474,
          0.03192737,  0.03321788],
        [-0.00573609,  0.03353577,  0.01554009, ..., -0.02407246,
         -0.02712448, -0.02615781],
        [-0.00055255, -0.00072391, -0.00272238, ..., -0.01496286,
         -0.02484781, -0.02799161]]], dtype=float32)>

**Note:** If we see the shape of the embedded sentence, we will notice it is (1, 15, 128)<br>
which means from every sentence we will take 15 words and each word will be represented with 128 shape matrix.

### `model_0`: Baseline

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [29]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("multinomial", MultinomialNB())
])

In [30]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('multinomial', MultinomialNB())])

### Evaluation function

In [31]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [32]:
def calculate_results(y_true, y_pred):
    """
    returns a dictionary of accuracy_score, precission, recall and f1_score
    """
    precission, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    acc_score = accuracy_score(y_true, y_pred) * 100
    
    results = {
        "accuracy_score": acc_score,
        "precission": precission,
        "recall": recall,
        "f1_score": f1_score
    }
    return results

In [33]:
baseline_results = calculate_results(val_labels, model_0.predict(val_sentences))
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_1`: Simple dense model

In [34]:
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
SAVE_DIR = "model_logs"

In [39]:
history_1 = model_1.fit(train_sentences, 
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                               experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230627-062221
Epoch 1/5
215/215 [==============================] - 18s 66ms/step - loss: 0.6389 - accuracy: 0.6513 - val_loss: 0.5746 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.4696 - accuracy: 0.8241 - val_loss: 0.4748 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3417 - accuracy: 0.8718 - val_loss: 0.4530 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2581 - accuracy: 0.9083 - val_loss: 0.4539 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1973 - accuracy: 0.9346 - val_loss: 0.4711 - val_accuracy: 0.7913


In [40]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.35415357],
       [0.85076755],
       [0.99511516],
       [0.05242171],
       [0.2636259 ],
       [0.97773355],
       [0.8716664 ],
       [0.9862852 ],
       [0.90453297],
       [0.09300441]], dtype=float32)

In [41]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
model_1_result = calculate_results(y_true=val_labels,
                                 y_pred=model_1_preds)
model_1_result

{'accuracy_score': 79.13385826771653,
 'precission': 0.7962095559621156,
 'recall': 0.7913385826771654,
 'f1_score': 0.788475062664411}

### `model_2`: LSTM

In [43]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [44]:
model_2.compile(loss="binary_crossentropy", 
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [45]:
history_2 = model_2.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "LSTM_model")])

Saving TensorBoard log files to: model_logs/LSTM_model/20230627-062303
Epoch 1/5
215/215 [==============================] - 18s 59ms/step - loss: 0.3682 - accuracy: 0.8367 - val_loss: 0.4908 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2433 - accuracy: 0.9060 - val_loss: 0.5486 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1733 - accuracy: 0.9362 - val_loss: 0.5740 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1295 - accuracy: 0.9510 - val_loss: 0.7387 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0929 - accuracy: 0.9635 - val_loss: 1.0786 - val_accuracy: 0.7651


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10] 

24/24 [==============================] - 1s 4ms/step


array([[8.9647304e-03],
       [8.8259977e-01],
       [9.9982709e-01],
       [2.1679904e-02],
       [7.7893952e-04],
       [9.9993789e-01],
       [9.1398239e-01],
       [9.9995267e-01],
       [9.9993646e-01],
       [4.9310410e-01]], dtype=float32)

In [47]:
model_2_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_result = calculate_results(y_true=val_labels,
                                  y_pred=model_2_pred)
model_2_result

{'accuracy_score': 79.13385826771653,
 'precission': 0.7962095559621156,
 'recall': 0.7913385826771654,
 'f1_score': 0.788475062664411}

In [49]:
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_3': GRU

In [50]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.GRU(units=64)(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU_model")

In [51]:
model_3.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [52]:
history_3 = model_3.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "GRU_model")])

Saving TensorBoard log files to: model_logs/GRU_model/20230627-062334
Epoch 1/5
215/215 [==============================] - 17s 57ms/step - loss: 0.1863 - accuracy: 0.9219 - val_loss: 0.6512 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1076 - accuracy: 0.9631 - val_loss: 0.7557 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0738 - accuracy: 0.9717 - val_loss: 1.0329 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0553 - accuracy: 0.9749 - val_loss: 1.3777 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0513 - accuracy: 0.9765 - val_loss: 1.2776 - val_accuracy: 0.7730


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:5]

24/24 [==============================] - 1s 4ms/step


array([[1.2150609e-03],
       [7.5226718e-01],
       [9.9984503e-01],
       [8.4675662e-02],
       [1.4372508e-04]], dtype=float32)

In [54]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [55]:
model_3_result = calculate_results(y_true=val_labels,
                                  y_pred=model_3_pred)
model_3_result

{'accuracy_score': 77.29658792650919,
 'precission': 0.7747736761679446,
 'recall': 0.7729658792650919,
 'f1_score': 0.7708425630155245}

### `model_4`: Bidirectional

In [56]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)
# x = layers.Bidirectional(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="bidirectional_model")

(None, 15, 128)
(None, 128)
(None, 128)


In [57]:
model_4.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [58]:
history_4 = model_4.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "bidirectional_model")])

Saving TensorBoard log files to: model_logs/bidirectional_model/20230627-062420
Epoch 1/5
215/215 [==============================] - 23s 68ms/step - loss: 0.1265 - accuracy: 0.9565 - val_loss: 0.8915 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0563 - accuracy: 0.9756 - val_loss: 1.3434 - val_accuracy: 0.7585
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0527 - accuracy: 0.9759 - val_loss: 1.2943 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0473 - accuracy: 0.9775 - val_loss: 1.5181 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0452 - accuracy: 0.9790 - val_loss: 1.2561 - val_accuracy: 0.7677


In [59]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:5]

24/24 [==============================] - 1s 5ms/step


array([[1.0274422e-03],
       [8.0738491e-01],
       [9.9961287e-01],
       [4.1187019e-03],
       [1.7942597e-04]], dtype=float32)

In [60]:
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [61]:
model_4_result = calculate_results(val_labels, model_4_pred)
model_4_result

{'accuracy_score': 76.77165354330708,
 'precission': 0.7715939358671309,
 'recall': 0.7677165354330708,
 'f1_score': 0.7645288433434009}

### `model_5`: Convolutional network

In [62]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
print(x.shape)
x = layers.GlobalMaxPool1D()(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

(None, 15, 128)
(None, 11, 32)
(None, 32)


In [63]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [64]:
model_5.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [65]:
history_5 = model_5.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230627-062509
Epoch 1/5
215/215 [==============================] - 16s 52ms/step - loss: 0.1734 - accuracy: 0.9556 - val_loss: 0.7529 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0807 - accuracy: 0.9726 - val_loss: 0.8977 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0645 - accuracy: 0.9762 - val_loss: 0.9807 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0575 - accuracy: 0.9778 - val_loss: 1.0150 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0517 - accuracy: 0.9785 - val_loss: 1.0617 - val_accuracy: 0.7612


In [66]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5]

24/24 [==============================] - 0s 2ms/step


array([[2.1212292e-01],
       [7.9336637e-01],
       [9.9998784e-01],
       [7.0342787e-02],
       [5.0985600e-06]], dtype=float32)

In [67]:
model_5_pred = tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [68]:
model_5_result = calculate_results(val_labels, model_5_pred)
model_5_result

{'accuracy_score': 76.11548556430446,
 'precission': 0.7622953452640989,
 'recall': 0.7611548556430446,
 'f1_score': 0.7591215732608759}

### `model_6`: Transfer learning

In [69]:
import tensorflow_hub as hub

In [70]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
#                                         input_shape=[],
#                                         dtype="string"
                                       )

In [71]:
# outputs =  layers.Dense(1, activation="sigmoid")(sentence_encoder_layer)
# model_6 = tf.keras.Model(sentence_encoder_layer, outputs)

In [74]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [78]:
model_6.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [80]:
history_6 = model_6.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "transfer_learning")])

Saving TensorBoard log files to: model_logs/transfer_learning/20230627-062704
Epoch 1/5
215/215 [==============================] - 8s 15ms/step - loss: 0.5006 - accuracy: 0.7901 - val_loss: 0.4651 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.4150 - accuracy: 0.8159 - val_loss: 0.4428 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4022 - accuracy: 0.8202 - val_loss: 0.4367 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3951 - accuracy: 0.8237 - val_loss: 0.4344 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.3865 - accuracy: 0.8288 - val_loss: 0.4334 - val_accuracy: 0.8084


In [81]:
model_6.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 64)                32832     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [82]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:5]

24/24 [==============================] - 1s 10ms/step


array([[0.17436613],
       [0.7848896 ],
       [0.98151785],
       [0.2047006 ],
       [0.73735315]], dtype=float32)

In [83]:
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 1.], dtype=float32)>

In [84]:
model_6_result = calculate_results(val_labels, model_6_pred)
model_6_result

{'accuracy_score': 80.83989501312337,
 'precission': 0.8096244700181708,
 'recall': 0.8083989501312336,
 'f1_score': 0.807165155491927}

In [85]:
model_1_result

{'accuracy_score': 79.13385826771653,
 'precission': 0.7962095559621156,
 'recall': 0.7913385826771654,
 'f1_score': 0.788475062664411}

### `model_7`: Transfer learning with 10% data

In [89]:
train_df_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [ ]:
# train_10_percent = train_df_shuffled[["text", "target"]].sample(frac=0.1, 
#                                                                 random_state=42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()


In [90]:
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentences),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
                                                                                                                            random_state=42)

In [86]:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [87]:
model_7.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [91]:
history_7 = model_7.fit(train_sentences_10_percent,
                       train_labels_10_percent,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "transfer_learning_10%_data")])

Saving TensorBoard log files to: model_logs/transfer_learning_10%_data/20230627-064221
Epoch 1/5
22/22 [==============================] - 3s 60ms/step - loss: 0.6686 - accuracy: 0.6880 - val_loss: 0.6454 - val_accuracy: 0.7139
Epoch 2/5
22/22 [==============================] - 1s 34ms/step - loss: 0.5935 - accuracy: 0.8105 - val_loss: 0.5893 - val_accuracy: 0.7454
Epoch 3/5
22/22 [==============================] - 1s 24ms/step - loss: 0.5156 - accuracy: 0.8134 - val_loss: 0.5348 - val_accuracy: 0.7651
Epoch 4/5
22/22 [==============================] - 1s 27ms/step - loss: 0.4527 - accuracy: 0.8324 - val_loss: 0.5027 - val_accuracy: 0.7690
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.4080 - accuracy: 0.8382 - val_loss: 0.4866 - val_accuracy: 0.7795


In [92]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:5]

24/24 [==============================] - 1s 10ms/step


array([[0.26101404],
       [0.80387735],
       [0.90884435],
       [0.32129198],
       [0.5406088 ]], dtype=float32)

In [93]:
model_7_pred = tf.squeeze(tf.round(model_7_pred_probs))
model_7_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 1.], dtype=float32)>

In [94]:
model_7_result = calculate_results(val_labels, model_7_pred)
model_7_result

{'accuracy_score': 77.95275590551181,
 'precission': 0.7833144267983451,
 'recall': 0.7795275590551181,
 'f1_score': 0.776722451043917}

In [95]:
model_1_result

{'accuracy_score': 79.13385826771653,
 'precission': 0.7962095559621156,
 'recall': 0.7913385826771654,
 'f1_score': 0.788475062664411}